# Artificial and Computational Intelligence Assignment 2

## Game Theory

List all the team members BITS ID ,Name along with % of contribution in this assignment: sample Provided below:
| BITS ID | Name | Contribution |
|---------|------|--------------|
| 2023aa05269 | ANWAR ABDU MOHAMED | 100% |
| 2022ac05722 | QURESHI SALEH HANIF | 100% |
| 2023aa05441 | ABHINAV SHUKLA | 100% |
| 2022ac05331 | VIGNESHWARAN K | 0% |

In [1]:
## Import library

import itertools
import math

def possible_choices(numbers, target):
    """Generates minimal subsets of 'numbers' that sum to at least 'target'.
    Args:
        numbers (set): A set of integers representing the available numbers.
        target (int): The minimum sum required for a valid subset.
    Returns:
        list: A list of sets, where each set is a minimal subset of 'numbers' that sums to at least 'target'.
              Minimal means no subset is a proper subset of another valid subset. """
 
    valid_choices = []
    # Generate all possible subsets of numbers with sums >= target
    for r in range(1, len(numbers) + 1):
        for subset in itertools.combinations(numbers, r):
            if sum(subset) >= target:
                valid_choices.append(set(subset))

    minimal_choices = []
    # Filter valid_choices to keep only minimal subsets
    for choice in valid_choices:
        is_minimal = True
        # Check if any other valid_choice is a subset of the current choice
        for other_choice in valid_choices:
            if choice != other_choice and other_choice.issubset(choice) and sum(other_choice) >= target:
                is_minimal = False
                break
        # If no other valid_choice is a subset, add it to minimal_choices
        if is_minimal:
            minimal_choices.append(choice)

    return minimal_choices

table_data = [] # Global list to store Minimax steps for table printing.

def print_table():
    """Prints the table of Minimax steps stored in 'table_data'. If 'table_data' is empty, the function does nothing. The table is formatted with 
    aligned columns, showing information like player turn, available numbers, choices, alpha/beta values, evaluation scores, and action notes."""
    
    if not table_data:
        return

    # Get the headers (column names) from the first row of table_data
    headers = list(table_data[0].keys())

    # Calculate the maximum width required for each column
    col_widths = {header: max(len(str(row.get(header, header))) for row in table_data) for header in headers}

    # Construct the header row with proper alignment
    header_row = " | ".join(f"{header:<{col_widths[header]}}" for header in headers)

    # Construct the separator row with hyphens
    separator_row = "-+-".join("-" * col_widths[header] for header in headers)

    # Print the header and separator rows
    print(header_row)
    print(separator_row)

    # Iterate through each row in table_data and print the data row
    for row in table_data:
        # Construct the data row with proper alignment
        data_row = " | ".join(f"{str(row[header]):<{col_widths[header]}}" for header in headers)
        print(data_row)
        
def minimax(numbers, p1_sum, p2_sum, is_p2_turn, alpha, beta, depth=0, indent=""):
    """ Implements the Minimax algorithm with Alpha-Beta pruning to find the best move.
    Args:
        numbers (set): The set of remaining numbers.
        p1_sum (int): The current score of Player 1.
        p2_sum (int): The current score of Player 2.
        is_p2_turn (bool): True if it's Player 2's turn, False if Player 1's turn.
        alpha (float): The alpha value for Alpha-Beta pruning.
        beta (float): The beta value for Alpha-Beta pruning.
        depth (int): The current depth of the recursion.
        indent (str): Used for visual indentation of the tree (not used in the table).
    Returns:
        tuple: A tuple containing the evaluation score and the best choice (subset of numbers).    """
    
    global table_data

    # Clear table_data at the start of the first minimax call (depth 0)
    if depth == 0:
        table_data = []

    # Base case: No numbers left, return the score
    if not numbers:
        score = p2_sum - p1_sum
        table_data.append({
            "Step": len(table_data) + 1,
            "Player": "P2" if is_p2_turn else "P1",
            "Available Numbers": numbers,
            "Choice": "-",
            "Alpha": alpha,
            "Beta": beta,
            "Evaluation Score": score,
            "Action/Notes": "Terminal node"
        })
        return score, set()

    best_choice = set()
    if is_p2_turn:  # Maximizing player (P2)
        max_eval = -math.inf
        # Iterate through all possible choices for P2
        for choice in possible_choices(numbers, p1_sum):
            remaining = numbers - choice
            # Recursive call to minimax for the next player (P1)
            eval_score, _ = minimax(remaining, p1_sum, p2_sum + sum(choice), False, alpha, beta, depth + 1, indent + "  ")
            # Update max_eval and best_choice if a better score is found
            if eval_score > max_eval:
                max_eval = eval_score
                best_choice = choice
            # Update alpha for pruning
            alpha = max(alpha, eval_score)
            # Pruning condition: If beta <= alpha, prune the branch
            if beta <= alpha:
                table_data.append({
                    "Step": len(table_data) + 1,
                    "Player": "P2",
                    "Available Numbers": numbers,
                    "Choice": "-",
                    "Alpha": alpha,
                    "Beta": beta,
                    "Evaluation Score": "-",
                    "Action/Notes": "Pruning (beta <= alpha)"
                })
                break
        # Record the best choice for P2 in table_data
        table_data.append({
            "Step": len(table_data) + 1,
            "Player": "P2",
            "Available Numbers": numbers,
            "Choice": best_choice,
            "Alpha": alpha,
            "Beta": beta,
            "Evaluation Score": max_eval,
            "Action/Notes": "P2 Choice" if depth == 0 else "P2 Return"
        })
        return max_eval, best_choice
    else:  # Minimizing player (P1)
        min_eval = math.inf
        # Iterate through all possible choices for P1
        for choice in possible_choices(numbers, p2_sum):
            remaining = numbers - choice
            # Recursive call to minimax for the next player (P2)
            eval_score, _ = minimax(remaining, p1_sum + sum(choice), p2_sum, True, alpha, beta, depth + 1, indent + "  ")
            # Update min_eval and best_choice if a better score is found
            if eval_score < min_eval:
                min_eval = eval_score
                best_choice = choice
            # Update beta for pruning
            beta = min(beta, eval_score)
            # Pruning condition: If beta <= alpha, prune the branch
            if beta <= alpha:
                table_data.append({
                    "Step": len(table_data) + 1,
                    "Player": "P1",
                    "Available Numbers": numbers,
                    "Choice": "-",
                    "Alpha": alpha,
                    "Beta": beta,
                    "Evaluation Score": "-",
                    "Action/Notes": "Pruning (beta <= alpha)"
                })
                break
        # Record the best choice for P1 in table_data
        table_data.append({
            "Step": len(table_data) + 1,
            "Player": "P1",
            "Available Numbers": numbers,
            "Choice": best_choice,
            "Alpha": alpha,
            "Beta": beta,
            "Evaluation Score": min_eval,
            "Action/Notes": "P1 Choice" if depth == 0 else "P1 Return"
        })
        return min_eval, best_choice
        
def select_best_subset(numbers, target, p1_sum, p2_sum):
    """ Selects the best subset for P2 using Minimax.
    Args:
        numbers (set): The set of remaining numbers.
        target (int): The target sum for P2 to reach or exceed.
        p1_sum (int): The current score of Player 1.
        p2_sum (int): The current score of Player 2.
    Returns:
        set: The best subset of numbers for Player 2 to choose. """
    
    # Initialize alpha and beta for Alpha-Beta pruning
    alpha, beta = -math.inf, math.inf

    # Call Minimax to get the best move and its evaluation score
    eval_score, best_move = minimax(numbers, p1_sum, p2_sum, True, alpha, beta)

    # Calculate the actual evaluation score for the chosen move
    if best_move:
        # If Minimax returned a best move, calculate the score based on that move
        actual_eval_score = p2_sum + sum(best_move) - p1_sum
    else:
        # If Minimax didn't return a move (e.g., no valid move found), use a fallback choice
        fallback_move = min(possible_choices(numbers, target), key=lambda x: (sum(x), len(x))) if possible_choices(numbers, target) else numbers.copy()
        actual_eval_score = p2_sum + sum(fallback_move) - p1_sum
        best_move = fallback_move

    # Append the final move and its details to the table_data for printing
    table_data.append({
        "Step": len(table_data) + 1,
        "Player": "P2",
        "Available Numbers": numbers,
        "Choice": best_move,
        "Alpha": alpha,
        "Beta": beta,
        "Evaluation Score": actual_eval_score,
        "Action/Notes": "P2 Final Choice"
    })

    # Print the table showing the Minimax steps
    print_table()

    # Return the best move (either from Minimax or the fallback)
    if best_move:
        return best_move
    else:
        valid_choices = possible_choices(numbers, target)
        if valid_choices:
            return min(valid_choices, key=lambda x: (sum(x), len(x)))
        return numbers.copy() if numbers else set()

def play_game(n):
    """Plays the Catch-Up Nim game with numbers from 1 to n.
    Args:
        n (int): The highest number in the initial set of numbers. """
    
    # Initialize the set of available numbers and player scores
    numbers = set(range(1, n + 1))
    p1_sum = 0
    p2_sum = 0

    # Print the initial game state
    print("Initial Numbers:", numbers)
    print(f"Possible choices for P1: {possible_choices(numbers, 0)}")

    # Player 1's first turn
    while True:
        try:
            # Get Player 1's choice from input
            p1_choice = set(map(int, input("P1, choose your numbers separated by spaces: ").split()))
            # Check if the choice is valid
            if p1_choice.issubset(numbers):
                break
            else:
                print("Invalid choice. Try again.")
        except ValueError:
            print("Invalid input. Enter numbers separated by spaces.")

    # Update the game state after Player 1's first turn
    numbers -= p1_choice
    p1_sum += sum(p1_choice)
    print(f"P1 chooses {p1_choice}. Remaining Numbers: {numbers}")

    # Main game loop
    while numbers:
        # Player 2's turn: use Minimax to select the best subset
        p2_choice = select_best_subset(numbers, p1_sum, p1_sum, p2_sum)
        numbers -= p2_choice
        p2_sum += sum(p2_choice)
        print(f"P2 chooses {p2_choice}. Remaining Numbers: {numbers}")

        # Check if the game is over
        if not numbers:
            break

        # Print possible choices for Player 1's next turn
        print(f"Possible choices for P1: {possible_choices(numbers, p2_sum)}")

        # Player 1's turn
        while True:
            try:
                # Get Player 1's choice from input
                p1_choice = set(map(int, input("P1, choose your numbers separated by spaces: ").split()))
                # Check if the choice is valid
                if p1_choice.issubset(numbers) and (sum(p1_choice) >= p2_sum or sum(numbers) < p2_sum):
                    break
                else:
                    print("Invalid choice. Try again.")
            except ValueError:
                print("Invalid input. Enter numbers separated by spaces.")

        # Update the game state after Player 1's turn
        numbers -= p1_choice
        p1_sum += sum(p1_choice)
        print(f"P1 chooses {p1_choice}. Remaining Numbers: {numbers}")

    # Game over: print the results
    print("Game Over!")
    print(f"Final Scores - P1: {p1_sum}, P2: {p2_sum}")
    if p1_sum > p2_sum:
        print("P1 Wins!")
    elif p1_sum < p2_sum:
        print("P2 Wins!")
    else:
        print("It's a Tie!")
        
# Get the highest number (n) from the user
n = int(input("Enter the highest number (n) to play with: "))

# Start the game with the given highest number
play_game(n)

Enter the highest number (n) to play with:  5


Initial Numbers: {1, 2, 3, 4, 5}
Possible choices for P1: [{1}, {2}, {3}, {4}, {5}]


P1, choose your numbers separated by spaces:  3


P1 chooses {3}. Remaining Numbers: {1, 2, 4, 5}
Step | Player | Available Numbers | Choice | Alpha | Beta | Evaluation Score | Action/Notes           
---+----+--------------+--------+------+------+------+------------------------
1  | P2 | {1, 2}       | set()  | -inf | inf  | -inf | P2 Return              
2  | P1 | {1, 2, 5}    | -      | -inf | -inf | -    | Pruning (beta <= alpha)
3  | P1 | {1, 2, 5}    | {5}    | -inf | -inf | -inf | P1 Return              
4  | P2 | {2}          | set()  | -inf | inf  | -inf | P2 Return              
5  | P1 | {1, 2, 4}    | -      | -inf | -inf | -    | Pruning (beta <= alpha)
6  | P1 | {1, 2, 4}    | {1, 4} | -inf | -inf | -inf | P1 Return              
7  | P2 | {5}          | set()  | -inf | inf  | -inf | P2 Return              
8  | P1 | {4, 5}       | -      | -inf | -inf | -    | Pruning (beta <= alpha)
9  | P1 | {4, 5}       | {4}    | -inf | -inf | -inf | P1 Return              
10 | P2 | {1, 2, 4, 5} | set()  | -inf | inf  | -inf | P2 C

P1, choose your numbers separated by spaces:  5


P1 chooses {5}. Remaining Numbers: {4}
Step | Player | Available Numbers | Choice | Alpha | Beta | Evaluation Score | Action/Notes   
--+----+-----+-------+------+-----+------+----------------
1 | P2 | {4} | set() | -inf | inf | -inf | P2 Choice      
2 | P2 | {4} | {4}   | -inf | inf | -1   | P2 Final Choice
P2 chooses {4}. Remaining Numbers: set()
Game Over!
Final Scores - P1: 8, P2: 7
P1 Wins!
